In [1]:
import pandas as pd
import mysql.connector

## Trip Dataset

### Are the most popular stations to start a trip also the most poppular station to end a trip?

In [ ]:
ax[0,0].plot(austin_weather['MONTH'], austin_weather["MLY-PRCP-NORMAL"], color = 'b')
ax[0,0].plot(austin_weather['MONTH'], austin_weather["MLY-PRCP-25PCTL"], color = 'b', linestyle = '--')
ax[0,0].plot(austin_weather['MONTH'], austin_weather["MLY-PRCP-75PCTL"], color = 'b', linestyle = '--')

In [2]:
cnx = mysql.connector.connect(user='root', password='ams_2020',host='127.0.0.1', database='practice_datasets')
cursor = cnx.cursor()

In [45]:
query = '''
    WITH 

        top_start AS (
        SELECT 
               start_station_name,
               COUNT(start_date) as count_start
            FROM trip
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 100),

        rank_start AS (
            SELECT top_start.*,
               RANK() OVER(ORDER BY top_start.count_start DESC) as rank_start
            FROM top_start),

        top_end AS (
        SELECT 
               end_station_name,
               COUNT(end_date) as count_end
            FROM trip
            GROUP BY end_station_name
            ORDER BY 2 DESC
            LIMIT 100),

        rank_end AS (
            SELECT top_end.*,
               RANK() OVER(ORDER BY top_end.count_end DESC) as rank_end
            FROM top_end)
        
        SELECT rs.*,
               re.*
            FROM rank_start AS rs
                LEFT JOIN rank_end AS re
                    ON rs.start_station_name=re.end_station_name
            ORDER BY rs.rank_start, re.rank_end;
'''

#query="SELECT * FROM trip LIMIT 10"

#cursor.execute(query)

In [46]:
df=pd.read_sql(query,cnx)

In [52]:
query=pd.read_sql('SELECT * FROM trip',cnx)
start = (query.groupby(['start_station_name'])
            ['start_date'].count()
            .reset_index()
            .rename(columns={'start_date':'count_start'})
            .sort_values(by='count_start',ascending=False)
        )
start['rank_start'] = start['count_start'].rank(ascending=False)

end = (query.groupby(['end_station_name'])
            ['end_date'].count()
            .reset_index()
            .rename(columns={'end_date':'count_end'})
            .sort_values(by='count_end',ascending=False)
        )
end['rank_end'] = end['count_end'].rank(ascending=False)

final = pd.merge(start,end,left_on='start_station_name',right_on='end_station_name',how='outer')
final = final.sort_values(by='rank_start')

In [53]:
final.head(10)

,start_station_name,count_start,rank_start,end_station_name,count_end,rank_end
0,San Francisco Caltrain (Townsend at 4th),49092,1.0,San Francisco Caltrain (Townsend at 4th),63179,1.0
1,San Francisco Caltrain 2 (330 Townsend),33742,2.0,San Francisco Caltrain 2 (330 Townsend),35117,2.0
2,Harry Bridges Plaza (Ferry Building),32934,3.0,Harry Bridges Plaza (Ferry Building),33193,3.0
3,Embarcadero at Sansome,27713,4.0,Embarcadero at Sansome,30796,4.0
4,Temporary Transbay Terminal (Howard at Beale),26089,5.0,Temporary Transbay Terminal (Howard at Beale),23080,9.0
5,2nd at Townsend,25837,6.0,2nd at Townsend,28529,5.0
6,Steuart at Market,24838,7.0,Steuart at Market,25025,8.0
7,Market at Sansome,24172,8.0,Market at Sansome,28033,6.0
8,Townsend at 7th,23724,9.0,Townsend at 7th,26637,7.0
9,Market at 10th,20272,10.0,Market at 10th,17864,12.0


In [49]:
df.head(10)

,start_station_name,count_start,rank_start,end_station_name,count_end,rank_end
0,San Francisco Caltrain (Townsend at 4th),49092,1,San Francisco Caltrain (Townsend at 4th),63179,1
1,San Francisco Caltrain 2 (330 Townsend),33742,2,San Francisco Caltrain 2 (330 Townsend),35117,2
2,Harry Bridges Plaza (Ferry Building),32934,3,Harry Bridges Plaza (Ferry Building),33193,3
3,Embarcadero at Sansome,27713,4,Embarcadero at Sansome,30796,4
4,Temporary Transbay Terminal (Howard at Beale),26089,5,Temporary Transbay Terminal (Howard at Beale),23080,9
5,2nd at Townsend,25837,6,2nd at Townsend,28529,5
6,Steuart at Market,24838,7,Steuart at Market,25025,8
7,Market at Sansome,24172,8,Market at Sansome,28033,6
8,Townsend at 7th,23724,9,Townsend at 7th,26637,7
9,Market at 10th,20272,10,Market at 10th,17864,12


In [35]:
query.rank(ascending=False)

,start_station_name,count_start
51,23.0,1.0
52,22.0,2.0
24,50.0,3.0
18,56.0,4.0
68,6.0,5.0
...,...,...
58,16.0,70.0
21,53.0,71.0
44,30.0,72.0
8,66.0,73.0
